In [1]:
from collections import defaultdict, Counter, OrderedDict

import nltk
import seaborn as sns
import os
import re
from gensim import corpora, models, similarities
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.tokenize import WordPunctTokenizer
from nltk.probability import FreqDist
from nltk.stem.snowball import PortugueseStemmer
import string
from string import punctuation
from pprint import pprint
from collections import Counter
from collections import defaultdict
import numpy as np
import math
from gensim.summarization.bm25 import BM25 
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Exercício 1
A partir de um corpus à sua escolha, estime um modelo de assuntos baseado no Modelo LSI. Uma vez calculado o modelo, defina um conjunto de documentos relevantes (${\cal R}$) para um assunto, como os $n$ documentos que contiverem em sua representação LSI, os maiores coeficientes para o assunto escolhido. Construa uma consulta $q$, com as dez palavras mais importantes do assunto escolhido.

In [2]:
textos = []
for p, d, f in os.walk(r'machado/machado'):
    #print( p,d,f)
    if f:
        for fileid  in f:
            if not fileid.endswith('.txt'):
                continue
            with open(os.path.join(p,fileid), encoding='iso-8859-1') as g:
                textos.append(g.read())
#textos = textos[:20]

In [3]:
wrong_symbols = ["", "\x97", "é", "ser", "\x93", "\x94", "--", "..."]
swu = stopwords.words('portuguese') + list (string.punctuation) + wrong_symbols
stemmer = PortugueseStemmer()
tok_textos = [WordPunctTokenizer().tokenize(t.lower()) for t in textos]

In [4]:
def clean_sentence(texto : str):
    return [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
textos_limpos = [*map(clean_sentence,textos)]

In [5]:
dicionario = corpora.Dictionary(textos_limpos)
corpus = [dicionario.doc2bow(d) for d in textos_limpos]
corpora.MmCorpus.serialize('machado.mm', corpus)
lsi = models.LsiModel(corpus, id2word=dicionario, num_topics=5) 

In [6]:
lsi.show_topics() 

[(0,
  '0.265*"outr" + 0.222*"a" + 0.214*"nã" + 0.201*"o" + 0.184*"tod" + 0.183*"pod" + 0.152*"cas" + 0.142*"diz" + 0.119*"algum" + 0.119*"diss"'),
 (1,
  '-0.296*"gilliatt" + -0.256*"diss" + -0.220*"mar" + -0.220*"o" + 0.207*"outr" + -0.142*"a" + -0.127*"era" + -0.113*"capítul" + 0.113*"pod" + -0.110*"lethierry"'),
 (2,
  '-0.321*"diss" + 0.282*"gilliatt" + 0.261*"o" + 0.187*"a" + -0.142*"mã" + -0.133*"d" + -0.129*"cas" + 0.123*"águ" + -0.122*"olhos" + -0.114*"rubiã"'),
 (3,
  '0.428*"sr" + 0.391*"oliv" + 0.321*"diss" + -0.210*"rubiã" + -0.196*"mar" + 0.158*"jud" + 0.139*"sik" + 0.131*"bumbl" + 0.130*"o" + -0.108*"sof"'),
 (4,
  '0.313*"rubiã" + 0.221*"oliv" + -0.218*"helen" + -0.206*"estáci" + -0.182*"a" + -0.168*"jorg" + 0.159*"mar" + 0.153*"sof" + -0.150*"moc" + -0.147*"coraçã"')]

In [7]:
first_topic = [doc[0][1] for doc in lsi[corpus]]
relevant_topic = sorted(((it, i) for i,it in enumerate(first_topic)), reverse = True)
relevant_topic = [(i,it) for it, i in relevant_topic][:12]
relevant_topic

[(232, 5171.286837587335),
 (210, 2167.8992126445182),
 (204, 1618.6188689714834),
 (202, 1536.0785187410888),
 (207, 1375.3599139114417),
 (230, 1299.989787555547),
 (201, 1163.1231753226741),
 (181, 1159.009605187778),
 (206, 1146.3011446363134),
 (209, 1145.5241441827798),
 (205, 1094.3474679694002),
 (199, 1093.824492313965)]

In [8]:
lsi.print_topic(0,10) 

'0.265*"outr" + 0.222*"a" + 0.214*"nã" + 0.201*"o" + 0.184*"tod" + 0.183*"pod" + 0.152*"cas" + 0.142*"diz" + 0.119*"algum" + 0.119*"diss"'

In [9]:
query = "Joao e Maria amor"
query_bow = dicionario.doc2bow(query.lower().split())
query_lsi = lsi[query_bow] 
query_lsi

[(0, 0.11292777749650365),
 (1, -0.055610302976167265),
 (2, -0.06659840225617694),
 (3, -0.016237808781098935),
 (4, -0.1492747332329445)]

## Exercício 2
Reutilizando os índices invertidos construídos em exercícios anteriores(Booleano, e TFIDF), calcule a precisão e revocação  com a consulta $q$ e o conjunto relevante ${\cal R}$ definidos no exercício anterior.

In [10]:
voc = set()
for text in textos_limpos:
    for word in text:
        voc.add(word)

### Consulta booleana

In [11]:
tok_textos = [WordPunctTokenizer().tokenize(t.lower()) for t in textos]
binary_matrix = np.zeros((len(tok_textos),len(query)))
for j, word in enumerate(query):
    for i, d in enumerate(tok_textos):
        binary_matrix[i,j] = int(word in d)

In [12]:
T = nltk.TextCollection(tok_textos)
matrix_tfidf = np.empty((len(tok_textos),len(query)))
for j, word in enumerate(query):
    for i, d in enumerate(tok_textos):
        matrix_tfidf[i,j] = T.tf_idf(word,d)

In [13]:
jaccard = dict()
for i, d in enumerate(tok_textos):
    intersec_set = set(query) & set(d)
    if intersec_set:
        jaccard[i] = len(intersec_set)/len(set(query) | set(d))
ordered_jaccard = OrderedDict(sorted(jaccard.items(), key=lambda k: k[1], reverse=True))
ordered_jaccard

OrderedDict([(245, 0.03409090909090909),
             (195, 0.02097902097902098),
             (243, 0.019867549668874173),
             (231, 0.013824884792626729),
             (37, 0.013513513513513514),
             (6, 0.01171875),
             (120, 0.011363636363636364),
             (2, 0.01090909090909091),
             (25, 0.00997506234413965),
             (194, 0.009925558312655087),
             (19, 0.009852216748768473),
             (222, 0.009230769230769232),
             (196, 0.00911854103343465),
             (191, 0.008982035928143712),
             (39, 0.008281573498964804),
             (229, 0.007894736842105263),
             (22, 0.007653061224489796),
             (16, 0.007537688442211055),
             (242, 0.007407407407407408),
             (23, 0.007058823529411765),
             (144, 0.00702576112412178),
             (65, 0.006944444444444444),
             (198, 0.006825938566552901),
             (91, 0.006787330316742082),
             (24, 0.0

In [14]:
def rev_precision(group, size=10):
    return len(set([relevante[0] for relevante in relevant_topic]) & set(list(group)[:size]))/ size

#### Revocação e precisão

In [15]:
rev_precision(ordered_jaccard.keys())

0.0

### TF-IDF

In [16]:
token_query = clean_sentence(query)
def normalize(v):
    if np.any(v):
        return v/norm(v)
    return v

def ordem(q):
    return [np.dot(normalize(q),normalize(r)) for r in matrix_tfidf]

v = np.array([T.tf_idf(word, token_query) for word in query])
r = ordem(v)
tfidf = dict((i,it) for i,it in enumerate(r[:len(tok_textos)]) if it != 0)

tfidf_sorted = OrderedDict(sorted(tfidf.items(), key=lambda v: v[1], reverse = True))

#### Revocação e precisão

In [17]:
rev_precision(tfidf_sorted.keys())

0.0

## Exercício 3
Usando as definições de probabilidade de relevância apresentadas no capítulo 11 do Livro, construa uma função de recuperação probabilística usando o log da razão de Odds como RSV (retrieval status value). Calcule revocação e precisão para consulta $q$ e conjunto relevante ${\cal R}$. Compare a probabilidade $p_t=P(x_t=1|R=1,q)$, com a o rankeamento de importância das palavras que compõem o assunto escolhido.

In [18]:
machado_gramatica = nltk.ConditionalFreqDist(nltk.bigrams(voc))
machado_gramatica_copia = nltk.ConditionalProbDist(machado_gramatica, nltk.MLEProbDist)

In [19]:
join_query = ' '.join(token_query)
cps = machado_gramatica_copia[join_query]
cps.samples()

dict_keys([])

## Exercício 4:
Repita o exercício 3 agora usando o modelo Okapi BM25 para o rankeamento. Calcule precisão e revocação.

In [20]:
bm_25 = BM25(textos_limpos)
average_idf = sum(float(bm_25.idf[key]) for key in bm_25.idf.keys()) / len(bm_25.idf)
average_idf

3.879946966825389

In [21]:
scores = bm_25.get_scores(join_query, average_idf)
rank_bm_25 = sorted(enumerate(scores), key = lambda x: x[1], reverse = True) 

#### Revocação e precisão;

In [22]:
rev_precision(rank_bm_25)

0.0